In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("DSL-StrongPasswordData.csv")

In [3]:
# 51 total
subjects = data["subject"].unique()

In [25]:
class euclideanDetector:
    
    def __init__(self, train, test_genuine, test_imposter):
        self.train = train
        self.test_genuine = test_genuine
        self.test_imposter = test_imposter
        self.user_scores = []
        self.imposter_scores = []
        self.mean_vector = []
        
    def training(self):
        self.mean_vector = self.train.mean().values
        
    def testing(self):
        for i in range(self.test_genuine.shape[0]):
            cur_score = np.linalg.norm(self.test_genuine.iloc[i].values - self.mean_vector)
            self.user_scores.append(cur_score)
            
        for i in range(self.test_imposter.shape[0]):
            cur_score = np.linalg.norm(self.test_imposter.iloc[i].values - self.mean_vector)
            self.imposter_scores.append(cur_score)
            
    def getScores(self):
        self.training()
        self.testing()
        return self.user_scores, self.imposter_scores

In [4]:
for subject in subjects:
    
    # Consider current subject as genuine and rest as imposters
    genuine_user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
    imposter_data = data.loc[data.subject != subject, :]
    
    # genuine user's first 200 time vectors for training
    train = genuine_user_data[:200]
    
    # True set (200 records)
    test_genuine = genuine_user_data[200:]
    
    # False set (250 records, 5 per imposter, 50 imposters in all)
    test_imposter = imposter_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]